In [ ]:
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import os
from pathlib import Path
from datetime import datetime
import re

def get_merged_cell_value(ws, row, col):
    """
    获取单元格的值，如果是合并单元格，返回合并区域左上角的值
    """
    cell = ws.cell(row=row, column=col)
    cell_value = cell.value
    
    # 如果单元格值为空，检查是否在合并单元格中
    if cell_value is None:
        cell_coordinate = cell.coordinate
        for merged_range in ws.merged_cells.ranges:
            if cell_coordinate in merged_range:
                # 获取合并区域左上角的单元格值
                top_left_cell = ws.cell(row=merged_range.min_row, column=merged_range.min_col)
                cell_value = top_left_cell.value
                break
    
    return cell_value

# 设置文件夹路径
threshold_folder = Path('Threshold')

# 存储所有数据的列表
all_data = []

# 获取所有 Excel 文件（排除临时文件）
excel_files = [f for f in threshold_folder.glob('*.xlsx') if not f.name.startswith('~$')]
excel_files.sort()  # 按文件名排序

print(f"找到 {len(excel_files)} 个文件")

# 处理每个文件
for file_path in excel_files:
    # 从文件名提取日期 (例如: 20171027_ThresholdHighlyCited.xlsx -> 20171027)
    filename = file_path.stem  # 获取不带扩展名的文件名
    date_match = re.match(r'(\d{8})', filename)
    
    if not date_match:
        print(f"警告: 无法从文件名 {filename} 提取日期，跳过")
        continue
    
    date_str = date_match.group(1)  # 20171027
    date_obj = datetime.strptime(date_str, '%Y%m%d')
    year = date_obj.year
    month = date_obj.month
    
    print(f"处理文件: {file_path.name}, 日期: {date_str}, 年份: {year}, 月份: {month}")
    
    # 使用 openpyxl 读取文件以处理合并单元格
    wb = load_workbook(file_path, data_only=True)
    ws = wb.active
    
    # 读取第3行获取年份列
    # 注意：只处理单元格本身有值的列，跳过合并单元格中的空列（如 D, K, O）
    header_row = 3
    year_columns = []
    
    # 读取第3行，只处理有值的单元格（跳过合并单元格中的空列）
    for col_idx in range(2, ws.max_column + 1):  # 从B列开始（跳过A列的研究领域）
        cell = ws.cell(row=header_row, column=col_idx)
        cell_value = cell.value
        
        # 只处理单元格本身有值的列（跳过合并单元格中的空列）
        if cell_value is not None:
            # 尝试转换为年份
            try:
                year_val = int(cell_value)
                if 2000 <= year_val <= 2100:  # 合理的年份范围
                    year_columns.append((col_idx, year_val))
            except:
                pass
    
    print(f"  找到年份列: {[y for _, y in year_columns]}")
    
    # 读取数据行（第4-25行）
    for row_idx in range(4, min(26, ws.max_row + 1)):  # 第4行到第25行
        # 获取研究领域名称（A列）
        research_field = get_merged_cell_value(ws, row_idx, 1)
        
        if research_field is None or research_field == '':
            continue
        
        # 为每个年份列读取对应的阈值数据
        for col_idx, year_val in year_columns:
            threshold_value = get_merged_cell_value(ws, row_idx, col_idx)
            
            # 添加到数据列表
            all_data.append({
                'RESEARCH FIELDS': research_field,
                'date': date_str,
                'Year': year_val,
                'Month': month,
                'Threshold-Highly Cited': threshold_value
            })
    
    wb.close()

print(f"\n总共处理了 {len(all_data)} 条记录")

# 创建 DataFrame
df = pd.DataFrame(all_data)

# 显示前几行
print("\n数据预览:")
print(df.head(10))

# 保存到 Excel 文件
output_file = 'Threshold_Combined.xlsx'
# 如果文件已存在，先删除
if os.path.exists(output_file):
    try:
        os.remove(output_file)
    except:
        pass

df.to_excel(output_file, index=False, engine='openpyxl')
print(f"\n数据已保存到: {output_file}")
print(f"总记录数: {len(df)}")
print(f"\n数据列: {df.columns.tolist()}")
print(f"\n日期范围: {df['date'].min()} 到 {df['date'].max()}")
print(f"年份范围: {df['Year'].min()} 到 {df['Year'].max()}")


找到 49 个文件
处理文件: 20171027_ThresholdHighlyCited.xlsx, 日期: 20171027, 年份: 2017, 月份: 10
  找到年份列: [2007, 2008, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2014, 2015, 2016, 2017, 2017]
处理文件: 20171217_ThresholdHighlyCited.xlsx, 日期: 20171217, 年份: 2017, 月份: 12
  找到年份列: [2007, 2008, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2014, 2015, 2016, 2017, 2017]
处理文件: 20180119_ThresholdHighlyCited.xlsx, 日期: 20180119, 年份: 2018, 月份: 1
  找到年份列: [2007, 2008, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2014, 2015, 2016, 2017, 2017]
处理文件: 20180315_ThresholdHighlyCited.xlsx, 日期: 20180315, 年份: 2018, 月份: 3
  找到年份列: [2007, 2008, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2014, 2015, 2016, 2017, 2017]
处理文件: 20180511_ThresholdHighlyCited.xlsx, 日期: 20180511, 年份: 2018, 月份: 5


C:\Users\ningh\scoop\apps\python\current\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


  找到年份列: [2008, 2009, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2015, 2016, 2017, 2018, 2018]
处理文件: 20180712_ThresholdHighlyCited.xlsx, 日期: 20180712, 年份: 2018, 月份: 7
  找到年份列: [2008, 2009, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2015, 2016, 2017, 2018, 2018]
处理文件: 20180913_ThresholdHighlyCited.xlsx, 日期: 20180913, 年份: 2018, 月份: 9
  找到年份列: [2008, 2009, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2015, 2016, 2017, 2018, 2018]
处理文件: 20181115_ThresholdHighlyCited.xlsx, 日期: 20181115, 年份: 2018, 月份: 11
  找到年份列: [2008, 2009, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2015, 2016, 2017, 2018, 2018]
处理文件: 20190118_ThresholdHighlyCited.xlsx, 日期: 20190118, 年份: 2019, 月份: 1
  找到年份列: [2008, 2009, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2015, 2016, 2017, 2018, 2018]


处理文件: 20190314_ThresholdHighlyCited.xlsx, 日期: 20190314, 年份: 2019, 月份: 3
  找到年份列: [2008, 2009, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2015, 2016, 2017, 2018, 2018]
处理文件: 20190509_ThresholdHighlyCited.xlsx, 日期: 20190509, 年份: 2019, 月份: 5
  找到年份列: [2009, 2010, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2016, 2017, 2018, 2019, 2019]
处理文件: 20190711_ThresholdHighlyCited.xlsx, 日期: 20190711, 年份: 2019, 月份: 7
  找到年份列: [2009, 2010, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2016, 2017, 2018, 2019, 2019]
处理文件: 20190911_ThresholdHighlyCited.xlsx, 日期: 20190911, 年份: 2019, 月份: 9
  找到年份列: [2009, 2010, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2016, 2017, 2018, 2019, 2019]
处理文件: 20200109_ThresholdHighlyCited.xlsx, 日期: 20200109, 年份: 2020, 月份: 1


  找到年份列: [2009, 2010, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2016, 2017, 2018, 2019, 2019]
处理文件: 20200312_ThresholdHighlyCited.xlsx, 日期: 20200312, 年份: 2020, 月份: 3
  找到年份列: [2009, 2010, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2016, 2017, 2018, 2019, 2019]
处理文件: 20200514_ThresholdHighlyCited.xlsx, 日期: 20200514, 年份: 2020, 月份: 5
  找到年份列: [2010, 2011, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2017, 2018, 2019, 2020, 2020]
处理文件: 20200709_ThresholdHighlyCited.xlsx, 日期: 20200709, 年份: 2020, 月份: 7
  找到年份列: [2010, 2011, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2017, 2018, 2019, 2020, 2020]
处理文件: 20200910_ThresholdHighlyCited.xlsx, 日期: 20200910, 年份: 2020, 月份: 9


...

  找到年份列: [2013, 2014, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2020, 2021, 2022, 2023, 2023]
处理文件: 20230915_ThresholdHighlyCited.xlsx, 日期: 20230915, 年份: 2023, 月份: 9
  找到年份列: [2013, 2014, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2020, 2021, 2022, 2023, 2023]
处理文件: 20231109_ThresholdHighlyCited.xlsx, 日期: 20231109, 年份: 2023, 月份: 11
  找到年份列: [2013, 2014, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2020, 2021, 2022, 2023, 2023]
处理文件: 20240123_ThresholdHighlyCited.xlsx, 日期: 20240123, 年份: 2024, 月份: 1
  找到年份列: [2013, 2014, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2020, 2021, 2022, 2023, 2023]
处理文件: 20240315_ThresholdHighlyCited.xlsx, 日期: 20240315, 年份: 2024, 月份: 3


  找到年份列: [2013, 2014, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2020, 2021, 2022, 2023, 2023]
处理文件: 20240510_ThresholdHighlyCited.xlsx, 日期: 20240510, 年份: 2024, 月份: 5
  找到年份列: [2014, 2015, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2021, 2022, 2023, 2024, 2024]
处理文件: 20240711_ThresholdHighlyCited.xlsx, 日期: 20240711, 年份: 2024, 月份: 7
  找到年份列: [2014, 2015, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2021, 2022, 2023, 2024, 2024]
处理文件: 20240912_ThresholdHighlyCited.xlsx, 日期: 20240912, 年份: 2024, 月份: 9
  找到年份列: [2014, 2015, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2021, 2022, 2023, 2024, 2024]
处理文件: 20241114_ThresholdHighlyCited.xlsx, 日期: 20241114, 年份: 2024, 月份: 11


  找到年份列: [2014, 2015, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2021, 2022, 2023, 2024, 2024]
处理文件: 20250109_ThresholdHighlyCited.xlsx, 日期: 20250109, 年份: 2025, 月份: 1
  找到年份列: [2014, 2015, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2021, 2022, 2023, 2024, 2024]
处理文件: 20250314_ThresholdHighlyCited.xlsx, 日期: 20250314, 年份: 2025, 月份: 3
  找到年份列: [2014, 2015, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2021, 2022, 2023, 2024, 2024]
处理文件: 20250508_ThresholdHighlyCited.xlsx, 日期: 20250508, 年份: 2025, 月份: 5
  找到年份列: [2015, 2016, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2022, 2023, 2024, 2025, 2025]
处理文件: 20250728_ThresholdHighlyCited.xlsx, 日期: 20250728, 年份: 2025, 月份: 7


  找到年份列: [2015, 2016, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2022, 2023, 2024, 2025, 2025]
处理文件: 20250919_ThresholdHighlyCited.xlsx, 日期: 20250919, 年份: 2025, 月份: 9
  找到年份列: [2015, 2016, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2022, 2023, 2024, 2025, 2025]
处理文件: 20251113_ThresholdHighlyCited.xlsx, 日期: 20251113, 年份: 2025, 月份: 11
  找到年份列: [2015, 2016, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2022, 2023, 2024, 2025, 2025]

总共处理了 15092 条记录

数据预览:
         RESEARCH FIELDS      date  Year  Month  Threshold-Highly Cited
0  AGRICULTURAL SCIENCES  20171027  2007     10                     129
1  AGRICULTURAL SCIENCES  20171027  2008     10                     111
2  AGRICULTURAL SCIENCES  20171027  2008     10                     111
3  AGRICULTURAL SCIENCES  20171027  2009     10                      93
4  AGRICULTURAL SCIENCES  20171027  2010     10                      89
5  AGRICULTURAL SCIENCES  20171027  2011     10                      69
6  AGRICULTURAL SCIENCES  20171027  2012     


数据已保存到: Threshold_Combined.xlsx
总记录数: 15092
